In [1]:
import random
from pynq import Overlay
gcd_overlay = Overlay("gcd_custom.bit")

In [2]:
gcd_overlay?

Type:            Overlay
String form:     <pynq.overlay.Overlay object at 0xaf376988>
File:            /usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/overlay.py
Docstring:      
Default documentation for overlay gcd_custom.bit. The following
attributes are available on this overlay:

IP Blocks
----------
gcd_custom_0         : pynq.overlay.DefaultIP
processing_system7_0 : pynq.overlay.DefaultIP

Hierarchies
-----------
None

Interrupts
----------
None

GPIO Outputs
------------
None

Memories
------------
PSDDR                : Memory
Class docstring:
This class keeps track of a single bitstream's state and contents.

The overlay class holds the state of the bitstream and enables run-time
protection of bindings.

Our definition of overlay is: "post-bitstream configurable design".
Hence, this class must expose configurability through content discovery
and runtime protection.

The overlay class exposes the IP and hierarchies as attributes in the
overlay. If no other drivers 

In [3]:
gcd_ip = gcd_overlay.gcd_custom_0

In [4]:
gcd_ip.register_map

RegisterMap {
  a = Register(a=write-only),
  b = Register(b=write-only),
  gcd = Register(gcd=0),
  gcd_ctrl = Register(gcd_ap_vld=1, RESERVED=0)
}

In [5]:
gcd_ip.register_map.gcd.address

32

In [6]:
gcd_ip.write(0x10,10) # 0x10 address = 16 = add_ip.register_map.a.address
gcd_ip.write(0x18,20) # 0x18 =24
gcd_ip.read(0x20) #gcd(10,20) = 10

10

In [7]:
gcd_ip.write(gcd_ip.register_map.a.address,10) # 0x10 address = 16 = add_ip.register_map.a.address
gcd_ip.write(gcd_ip.register_map.b.address,20) # 0x18 =24
gcd_ip.read(gcd_ip.register_map.gcd.address)

10

In [16]:
#hw : gcd calculator에 대해서 이런 driver 만드는것!
class gcd_overlay(Overlay):
    def get_gcd(self, a,b):
        gcd_ip = self.gcd_custom_0
        gcd_ip.write(gcd_ip.register_map.a.address,a) # 0x10 address = 16 = add_ip.register_map.a.address
        gcd_ip.write(gcd_ip.register_map.b.address,b) # 0x18 =24
        return gcd_ip.read(gcd_ip.register_map.gcd.address)

In [17]:
hw = gcd_overlay('gcd_custom.bit')

In [20]:
rand_a = random.randrange(2,5000)
rand_b = random.randrange(2,5000)

def gcd_sw(a,b):
    if(b>a):
        c = a
        a = b
        b = c

    while(b):
        a, b = b, a%b
        
    return a

print(rand_a, rand_b)
print(hw.get_gcd(rand_a,rand_b))

print(rand_a, rand_b)
print(gcd_sw(rand_a,rand_b))

assert(gcd_sw(rand_a,rand_b) == hw.get_gcd(rand_a,rand_b))


248 4667
1
248 4667
1
